# Lesson3 系列データで分類・予測させてみよう（RNN, LSTM）

## 目次

- Section3 精度向上Tips
    - 3.1 Clippingによる勾配爆発への対処
    - 3.2 ショートカットとゲートによる勾配消失への対処
    - 3.3 LSTM
    - 3.4 GRU
    - Checkクイズ

## Section3 精度向上Tips

### 3.1 Clippingによる勾配爆発への対処

先程§1において、SimpleRNNの課題として勾配爆発と勾配消失について扱いました。

このうち勾配爆発については**Gradient Clipping**と呼ばれる、勾配の大きさそのものを制限してしまうという手法が有効です。

![Clipping](figures/Clipping.png)
出典：Ian Goodfellow et. al, “Deep Learning”, MIT press, 2016 (http://www.deeplearningbook.org/)

この図のグリッドが損失関数のパラメータに対する振る舞いを表しています。勾配は各点でのグリッド面の傾きに対応します。

学習に当たっては損失関数が最小となる点を探したいわけですが、こうした急峻な形状の損失関数では、「崖」に相当する部分で勾配爆発が発生します。

Clippingを行わない場合、極端な例では上図の左のように、「崖」での勾配爆発で探索点を一気に押し戻され、それまでの探索が無駄になってしまいます。

そこで、Clippingを行うことで上図右のように、過大な勾配については制限し、探索点が過剰に変化することを抑制しています。

ただし、あまりに制限値を小さくとると常に勾配が小さくなり、学習のスピードが落ちる点には注意を要します。

KerasでGradient Clippingを行うには、Optimizerを指定する際に引数として`clipnorm`または`clipvalue`を指定します。

```python
from tensorflow.keras import optimizers

sgd = optimizers.SGD(lr=0.01, clipnorm=1.)    # clipnormは勾配の2乗ノルムの最大値を制限する
sgd2 = optimizers.SGD(lr=0.01, clipvalue=1.)  # clipvalueは勾配の"要素"の絶対値の大きさを制限する
# clipnormの方が勾配の方向を変えないという利点があるが、経験的にはどちらの振る舞いも大差ない

ada = optimizers.Adagrad(lr=0.01, clipnorm=1.)# SGDに限らずすべてのoptimizerで指定可能

#model.compile(loss='mean_squared_error', optimizer=sgd)
```

---
なお、2乗ノルム$|| \cdot ||_2$とは、

* ベクトルに対する、通常の距離（ベクトル$v$に対して$||v||_2=\sqrt{v_1^2+v_2^2+\dots+v_n^2}$）
* 行列に対する、Frobeniusノルム（行列$A$に対して$||A||_2=\sqrt{\sum_{i=1}^{m}\sum_{j=1}^{n}a_{ij}^2}$）

のことを表します。

### 3.2 ショートカットとゲートによる勾配消失への対処

Clippingは勾配爆発への対処として有力ですが、勾配消失への対処としては**ショートカット**と呼ばれる手法が効果を発揮します。

これは、各層の出力にその層への入力だったものも加えてしまうという手法です。

（RNNに限らず、一般に第$l$層目の出力を$o^{(l)}=f^{(l)}(o^{(l-1)})+o^{(l-1)}$とすることで可能）

<img src="figures/shortcut.png" width=50%>

一見するとこれが勾配消失に有用なのか疑わしいですが、このショートカットよりこの層の勾配が**「1+元の勾配」**と増加します。

そのため、勾配の積が積み重なる、入口に近い層でも勾配が消失することなく、パラメータ更新が可能になります。

---

また、このショートカットの概念に加え、**ゲート**という概念も重要です。

これは、ショートカットの一般化として重み付き和を考えるものです。

（つまり、$f^{(l)}(o^{(l-1)})$と$o^{(l-1)}$に各々係数をかけたうえで足し合わせる）

<img src="figures/gate.png" width=50%>

なお図中の$\odot$は要素ごとの積（アダマール積）を表します。

この**ゲートの係数も学習**することにより、前の層からの情報と現在の層による情報の重みを最適に調整できる、

つまり以前の層からの情報の忘却度合いを丁度よく決められます。

---

ここまでのテクニックはRNNに限らず、一般のDNNの層を重ねる際に使用できます。

RNNの場合は、特にこのゲートを（層方向だけでなく）時間方向に適用することで長期記憶や短期記憶を実現した、**ゲート付きRNN**が重要になります。

次節以降では、このゲート付きRNNとして代表的なLSTMとGRUを紹介します。

### 3.4 LSTM

**Long Short Term Memory（LSTM）**はゲートの考え方を時間方向の隠れ状態の計算に用いることで、

系列内の長期的な相互依存性をモデル化できるようにしたRNNで、頻繁に使用されるモデルの1つです。

<img src="figures/lstm.png" width=50%>

上図がLSTMの時点$t$でのモデルの模式図となります。（こういった素子が横に系列長$T$だけ並んでいます）

LSTMでは時点$t$の入力情報（時点$t$の入力$x_t$、前時点での出力$h_{t-1}$を結合したもの）を入力ゲート$i_t$を介して取り込んでいます。

この入力情報を用いて、前時点までの長期的な系列情報$c_{t-1}$を$c_t$に更新します。

なお、このとき前時点までの情報$c_{t-1}$をどれだけ重視するか、忘却ゲート$f_t$が制御しています。

最終的な出力は、時点tまでの系列情報$c_t$を出力ゲート$o_t$によって調整することで決定されます。

これらを線形変換や活性化関数を含めてしっかりと数式で表現した場合、以下のようになります。（必ずしも厳密に理解しなくても問題ありません）

$$
\begin{align*}
\bar{h}_t &= \mathrm{tanh} \left( W_\bar{h} x_t + R_\bar{h} h_{t-1} + b_\bar{h} \right)\\
c_t &= i_t \odot \bar{h}_t + f_t \odot c_{t-1}\\
h_t &= o_t \odot \mathrm{tanh}(c_t)
\end{align*}
$$

そして、各ゲートの係数$i_t, f_t, o_t$は以下のようにして、時点$t$の入力情報を元に決定されます。

パラメータ$W, R, b$を学習により決定することで、ゲート係数も多様なデータに柔軟に対応できます。

$$
\begin{align*}
i_t &= \sigma \left( W_i x_t + R_i h_{t-1} + b_i \right)\\
f_t &= \sigma \left( W_f x_t + R_f h_{t-1} + b_f \right)\\
o_t &= \sigma \left( W_o x_t + R_o h_{t-1} + b_o \right)
\end{align*}
$$

ここまで詳しくLSTMの構造を見てきましたが、Kerasにおいてはそうした詳細を気にする必要はほとんどありません。

というのも、次の`keras.layers.LSTM`を用いるだけで利用できるためです。

むしろ大事な点は、RNNの中でも長い系列に強い（系列内の長期的な相互依存性をモデル化可能）モデルであるという特性を認識しておくことです。

https://keras.io/ja/layers/recurrent/#lstm

```python
keras.layers.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                  kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', 
                  unit_forget_bias=True, 
                  kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
                  kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
                  dropout=0.0, recurrent_dropout=0.0, implementation=1, 
                  return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False)
```

引数は次の通りです。

* units：ユニット数（系列長$T$）
* activation：活性化関数
* recurrent_activation：ゲート係数の計算で使用する活性化関数
* use_bias：バイアスベクトル（$Wx_t+Rh_{t-1}$に付け加えるベクトル）を使用するか
* {kernel,recurrent,bias}_initializer：各パラメータの初期化法（kernelは$W$, recurrentは$R$を指す）
* unit_forget_bias：忘却ゲートを1に初期化
* {kernel,recurrent,bias,activity}_regularizer：各パラメータの正則化（activityは出力=activationを指す）
* {kernel,recurrent,bias}_constraint：各パラメータに課す制約
* dropout：入力についてのdropoutの比率（$W$に対するdropout）
* recurrent_dropout：再帰についてのdropoutの比率（$R$に対するdropout）
* return_sequences: Falseなら出力としては系列の最後の出力のみ（$o_T$のみ）を返す、Trueなら出力として完全な系列（$o_1,o_2,\ldots,o_T$）を返す
* return_state: Trueのときは出力とともに，最後の状態（$s_T$）を返す
* go_backwards: Trueのときは入力系列を後ろから処理する（出力も逆順に）
* stateful: Trueのときは、前バッチの各サンプルに対する最後の状態を、次のバッチのサンプルに対する初期状態として引き継ぐ
* unroll: （高速化のためのオプション）Trueのときは再帰が展開され高速化されるが、よりメモリに負荷がかかる（短い系列にのみ適する）

### 3.3 GRU

**Gated Recurrent Unit（GRU）**は、ゲートの考え方を利用しながら、隠れ状態ベクトル$h_t$のみに長期の情報も集約したモデルです。

（LSTMでは、長期の状態と短期の状態を$c_t,h_t$の2本で管理していました）

<img src="figures/gru.png" width=50%>

このように、状態ベクトルは$h_t$のみ、ゲートも$r_t, z_t$の2つのみで、LSTMと比較して（同じ系列長なら）計算量も使用空間量も少なく済みます。

（ただし、RNNとしての性能については、タスクごとにLSTMとGRUのどちらが有意となるかは違っており、甲乙つけがたいという状況です）

GRUの実装もKerasでは簡単に行うことができ、次の`keras.layers.GRU`を用います。

https://keras.io/ja/layers/recurrent/#gru

```python
keras.layers.GRU(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                 kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', 
                 kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
                 kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
                 dropout=0.0, recurrent_dropout=0.0, implementation=1, 
                 return_sequences=False, return_state=False, go_backwards=False, stateful=False, unroll=False, reset_after=False)
```

代表的な引数は次の通りです。

* units：ユニット数（系列長$T$）
* activation：活性化関数
* recurrent_activation：内部で使用する活性化関数
* use_bias：バイアスベクトル（$Ux_t+Wh_{t-1}$に付け加えるベクトル）を使用するか
* {kernel,recurrent,bias}_initializer：各パラメータの初期化法（kernelは上図$U$, recurrentは上図$W$を指す）
* {kernel,recurrent,bias,activity}_regularizer：各パラメータの正則化（activityは出力=activationを指す）
* {kernel,recurrent,bias}_constraint：各パラメータに課す制約
* dropout：入力についてのdropoutの比率
* recurrent_dropout：再帰についてのdropoutの比率（上図横矢印に対して適用するdropout）
* return_sequences: Falseなら出力としては系列の最後の出力のみ（$o_T$のみ）を返す、Trueなら出力として完全な系列（$o_1,o_2,\ldots,o_T$）を返す
* return_state: Trueのときは出力とともに，最後の状態（$h_T$）を返す
* go_backwards: Trueのときは入力系列を後ろから処理する（出力も逆順に）
* stateful: Trueのときは、前バッチの各サンプルに対する最後の状態を、次のバッチのサンプルに対する初期状態として引き継ぐ
* unroll: （高速化のためのオプション）Trueのときは再帰が展開され高速化されるが、よりメモリに負荷がかかる（短い系列にのみ適する）

### Checkクイズ

* 問題1
    * BPTTの説明として最も正しいものを選択しなさい
* 選択肢
  1. BPTTは時間軸方向に誤差が伝播されることを表現している
  2. BPTTはRNNにおける学習則である
  3. BPTTを用いると入力付近の誤差が過大に、出力付近の誤差が過小になる傾向がある
  4. LSTMではBPTTは行われない


* 問題2
    * 勾配爆発や勾配消失の説明として誤っているものを選びなさい
* 選択肢
  1. 勾配爆発問題はGradient Clippingによって対処できる
  2. RNNの入力付近では勾配消失が、出力付近では勾配爆発が問題となる
  3. 勾配爆発が起きると、学習が素早く終了するため、積極的に利用するべきである
  4. 勾配消失問題はLSTMの使用によってある程度対処できる
  
  
* 問題3
    * ゲートやショートカットの説明として最も正しいものを選択しなさい
* 選択肢
  1. ショートカットは勾配消失問題への対処策の1つである
  2. ショートカットは、活性化関数を通さずに次の層に直接出力を渡すことである
  3. ゲートはショートカットの特殊化である
  4. ゲートの重みは事前にハイパーパラメータとして決めておく
  
  
* 問題4
    * LSTMの説明として最も正しいものを選択しなさい
* 選択肢
  1. LSTMは入力、記憶、忘却、出力の4つのゲートを使用している
  2. LSTMの隠れ状態ベクトルは文脈ベクトルと呼ばれる
  3. LSTMは一般に短い時系列データよりも長い時系列データに対してより良い結果をもたらす
  4. 一般に長い時系列データに対してはLSTMの方が単純なRNN（kerasでのSimpleRNN）よりも良い結果をもたらす

### 参考文献

RNNに関して参考となる書籍をいくつか挙げておきますので、他の説明やより詳しい内容の情報源として利用してみてください。

1. 瀧雅人, "これならわかる深層学習入門", 2017, 講談社 \[[amazon](https://www.amazon.co.jp/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%82%B9%E3%82%BF%E3%83%BC%E3%83%88%E3%82%A2%E3%83%83%E3%83%97%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-%E3%81%93%E3%82%8C%E3%81%AA%E3%82%89%E3%82%8F%E3%81%8B%E3%82%8B%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E5%85%A5%E9%96%80-KS%E6%83%85%E5%A0%B1%E7%A7%91%E5%AD%A6%E5%B0%82%E9%96%80%E6%9B%B8-%E7%80%A7-%E9%9B%85%E4%BA%BA/dp/4061538284)\]
  - Chapter 9.にRNNについての説明があります。 数式は多めかもしれませんが、RNNのイメージについてもわかりやすく解説されています。
  
1. I. Goodfellow et al. "Deep Learning ", 2016, MIT Press \[[オンライン版(無料)](http://www.deeplearningbook.org/)\], \[[amazon(原著)](https://www.amazon.co.jp/Deep-Learning-Adaptive-Computation-Machine/dp/0262035618)\], \[[amazon(訳本)](https://www.amazon.co.jp/%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92-Ian-Goodfellow/dp/4048930621)\]
  - Chapter 10.にRNNについて詳細な記述があります. 上述の書籍より, 数式的なハードルは低く, さらに広範かつ詳細に記述されている印象です。英語に抵抗がない方は読んでみるとよいかもしれません。（あるいは訳本を参照してください）

1. 坪井雄太 他, "深層学習による自然言語処理", 2017, 講談社 \[[amazon](https://www.amazon.co.jp/%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AB%E3%82%88%E3%82%8B%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%83%95%E3%82%A7%E3%83%83%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-%E5%9D%AA%E4%BA%95-%E7%A5%90%E5%A4%AA/dp/4061529242)\]
  - この書籍自体は次回扱う自然言語処理を中心に扱っていますが、その基礎としてChapter 2.にRNNについての説明があります。自然言語処理に興味がある人はこの本でRNNを学んでもよいかもしれません。